In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

In [12]:
await atdomtraj.cmd_enable.start()

RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 15 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 15 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 32 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 11 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 14 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 15 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 11 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages


In [8]:
await atdome.cmd_moveAzimuth.set_start(azimuth=120.)

# ATMCS

In [51]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, summaryState) falling behind; read 34 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 22 elements
RemoteEvent(ATMCS, 0, atMountState) falling behind; read 34 messages


Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

# ATPtg

In [52]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

Done


[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [36]:
await atptg.cmd_pointLoadModel.set_start(pointingFile='/home/saluser/auxtel_20190911.mod')

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [45]:
pt = await atptg.evt_pointingModel.next(flush=False, timeout=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


TimeoutError: 

In [44]:
print(pt.pointingModelTermNames)
print(pt.pointingModelTermValues)

IA,IE,NPAE,CA,AW,AN
+2315.0782   185.68168,-1697.4554     6.33223,+98.4388   271.96853,-244.5659   323.01660,+252.5971     6.73688,+42.9223     7.07061


In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=243., elDegs=80.)

In [9]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [10]:
ra = Angle("20:25:38.85705", unit=u.hour)
dec = Angle("-56:44:06.3230", unit=u.deg)
target_name="Alf PAv"
radec = ICRS(ra, dec)

In [53]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteTelemetry(ATDome, 1, position) falling behind; read 21 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 21 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 20 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 20 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 20 messages


58740.01938958563
slew...


In [47]:
await atptg.cmd_stopTracking.start(timeout=10)

RemoteEvent(ATDome, 1, heartbeat) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 93 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATAOS, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 ele

In [48]:
await atmcs.cmd_stopTracking.start()

RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 16 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 20 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 20 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 20 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue co

In [ ]:
ss = await atdome.evt_summaryState.next(flush=False, timeout=5)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
ss = await atdomtraj.evt_summaryState.next(flush=False, timeout=5)

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await atdomtraj.cmd_enable.start()

In [ ]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [ ]:
dalt_grid = np.arange(-0.25, +0.255, 0.05)+0.45
daz_grid = np.arange(-0.25, +0.255, 0.05)-0.4

In [21]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-100., 
                                         num=0)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 20 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 41 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 41 messages


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await atptg.cmd_pointNewFile.start()

In [ ]:
await atptg.cmd_pointAddData.start()

In [ ]:
await atptg.cmd_pointNewFile.start()
await atptg.cmd_pointAddData.start()
await atptg.cmd_pointCloseFile.start()

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                         az=grid_az[i]*60*60, 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-20, 
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., 
                                     az=0, 
                                         num=1)

In [ ]:
60-61

In [ ]:
await asyncio.sleep(5)
off = await atptg.tel_guidingAndOffsets.next(flush=True, timeout=5)

In [ ]:
print(off)

In [25]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 33 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 75 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 33 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 74 elements
falling behind; queue contains 73 elements
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 25 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 33 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 33 mes

In [24]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35+0.5, u=0.35, v=0.22)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 78 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) fallin